In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [64]:
site_visits_df = pd.read_csv('../data/ECHO_data_csv/SDWA_SITE_VISITS.csv', dtype=str)

In [65]:
site_visits_df.head()

,SUBMISSIONYEARQUARTER,PWSID,VISIT_ID,VISIT_DATE,AGENCY_TYPE_CODE,VISIT_REASON_CODE,MANAGEMENT_OPS_EVAL_CODE,SOURCE_WATER_EVAL_CODE,SECURITY_EVAL_CODE,PUMPS_EVAL_CODE,OTHER_EVAL_CODE,COMPLIANCE_EVAL_CODE,DATA_VERIFICATION_EVAL_CODE,TREATMENT_EVAL_CODE,FINISHED_WATER_STOR_EVAL_CODE,DISTRIBUTION_EVAL_CODE,FINANCIAL_EVAL_CODE,VISIT_COMMENTS,FIRST_REPORTED_DATE,LAST_REPORTED_DATE
0,2021Q2,IL3125096,#2,08/05/1992,NaN,SNSV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,07/22/1995,NaN
1,2021Q2,MN1500006,1038,02/04/2003,ST,SNSV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12/06/2005,06/27/2012
2,2021Q2,MN1500006,19079,09/09/2004,ST,SNSV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12/06/2005,06/27/2012
3,2021Q2,MN1500007,1040,01/27/2003,ST,SNSV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12/06/2005,06/27/2012
4,2021Q2,MN1500007,19080,09/29/2004,ST,SNSV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12/06/2005,06/27/2012


In [66]:
site_visits_df.shape

(2154196, 20)

In [67]:
site_visits_df.isnull().sum()

SUBMISSIONYEARQUARTER                  0
PWSID                                  0
VISIT_ID                               0
VISIT_DATE                            15
AGENCY_TYPE_CODE                  440842
VISIT_REASON_CODE                      0
MANAGEMENT_OPS_EVAL_CODE          761619
SOURCE_WATER_EVAL_CODE            762129
SECURITY_EVAL_CODE               1447682
PUMPS_EVAL_CODE                   763791
OTHER_EVAL_CODE                   998389
COMPLIANCE_EVAL_CODE              762083
DATA_VERIFICATION_EVAL_CODE       761908
TREATMENT_EVAL_CODE               762783
FINISHED_WATER_STOR_EVAL_CODE     762062
DISTRIBUTION_EVAL_CODE            761853
FINANCIAL_EVAL_CODE              1415258
VISIT_COMMENTS                   1587661
FIRST_REPORTED_DATE                    0
LAST_REPORTED_DATE                477666
dtype: int64

In [80]:
## Seeing date formats
date_columns = ['VISIT_DATE', 'FIRST_REPORTED_DATE', 'LAST_REPORTED_DATE']
site_visits_df[date_columns].head()

,VISIT_DATE,FIRST_REPORTED_DATE,LAST_REPORTED_DATE
0,08/05/1992,1995-07-22,NaT
1,02/04/2003,2005-12-06,2012-06-27
2,09/09/2004,2005-12-06,2012-06-27
3,01/27/2003,2005-12-06,2012-06-27
4,09/29/2004,2005-12-06,2012-06-27


In [81]:
for datecol in date_columns:
    site_visits_df[datecol] = pd.to_datetime(site_visits_df[datecol], format='%m/%d/%Y')

In [82]:
site_visits_df[date_columns].head()

,VISIT_DATE,FIRST_REPORTED_DATE,LAST_REPORTED_DATE
0,1992-08-05,1995-07-22,NaT
1,2003-02-04,2005-12-06,2012-06-27
2,2004-09-09,2005-12-06,2012-06-27
3,2003-01-27,2005-12-06,2012-06-27
4,2004-09-29,2005-12-06,2012-06-27


In [83]:
for datacol in date_columns:
    print('Range of {} is between {} and {}'.format(datacol, site_visits_df[datacol].min(), site_visits_df[datacol].max()))

Range of VISIT_DATE is between 1900-01-01 00:00:00 and 2021-06-29 00:00:00
Range of FIRST_REPORTED_DATE is between 1995-07-22 00:00:00 and 2021-07-01 00:00:00
Range of LAST_REPORTED_DATE is between 2005-11-15 00:00:00 and 2021-07-01 00:00:00


In [38]:
# site_visits_df.VISIT_ID.str.replace('#', '')

In [91]:
site_visits_df[site_visits_df.VISIT_COMMENTS.str.len() > 1000]  #.tolist()

,SUBMISSIONYEARQUARTER,PWSID,VISIT_ID,VISIT_DATE,AGENCY_TYPE_CODE,VISIT_REASON_CODE,MANAGEMENT_OPS_EVAL_CODE,SOURCE_WATER_EVAL_CODE,SECURITY_EVAL_CODE,PUMPS_EVAL_CODE,OTHER_EVAL_CODE,COMPLIANCE_EVAL_CODE,DATA_VERIFICATION_EVAL_CODE,TREATMENT_EVAL_CODE,FINISHED_WATER_STOR_EVAL_CODE,DISTRIBUTION_EVAL_CODE,FINANCIAL_EVAL_CODE,VISIT_COMMENTS,FIRST_REPORTED_DATE,LAST_REPORTED_DATE
1334,2021Q2,NJ0717001,11,2015-04-02,ST,SNSV,N,N,NaN,N,NaN,N,N,N,N,N,NaN,"1. Community potable water system number PW0717001. 2. Orange must provide public notice as required by the State regulations for exceedances during 2014 of the recommended upper limits for sodium. Facility provides Tier 3 public notice in the CCR that is mailed out every year to customers. 3. There is an out of service well (Brook Alley well) located by the Department of Public Works garage. This well is not listed in the Water Allocation Permit 5234. If an inactive well has not been in operation for three or more years, N.J.A.C. 7:19-1.6(a)3. requires sealing of a well in accordance with N.J.S.A. 58:4A-4.1 et. seq. and N.J.A.C. 7:9-9. Also, N.J.A.C. 7:10-12.16(g) requires decommissioning of a well in accordance with N.J.A.C. 7:9 if a well cannot be used for its intended use. The Orange Water Department submitted correspondence on August 17, 2010, stating that the Water Department wished to keep the well active. 4. The new pumping / treatment station at Beech Street is constructed and is running. There is a new emergency generator on site. According to personnel, the emergency generator is exercised on a regular basis.",2015-08-24,2021-07-01
2620,2021Q2,NC0392335,13,2015-06-11,ST,SNSV,N,M,N,N,N,N,N,N,M,N,N,"LORRIE STAGNER~ 1.SYSTEM IS WELL MAINTAINED AND IN GOOD CONDITION. 2. SYSTEM REQUIRES OPERATOR VISIT TREATMENT PLANT P01, P04 AND P05 1X/WK DUE TO OUT SYSTEM. P03 REQUIRES 2X/WK VISITS. DISTRIBUTION REQUIRES 3X/WK CHLORINE RESIDUAL MEASUREMENT. 3.MONTHLY OPERATING REPORTS ARE SUBMITTED. 4.SOURCE CONSISTS OF FIVE WELLS. WH1 AND DD1 ARE OFFLINE AT THE BREAKER AND NOT IN USE. WS2 IS LISTED AS AVAILABILITY &quot;OTHER&quot;. SF1 AND WS1 ARE ACTIVE AND PROVIDE SUFFICIENT CAPACITY FOR THE SYSTEM. THE CHECK VALVE AT SF1 WAS LEAKING. WH1 WAS 50% THROTTLED. NO OTHER ISSUES NOTED. 5. TREATMENT CONSISTS OF FIVE TREATMENT PLANTS. APPROPRIATE APPROVED TREATMENT WAS INSTALLED AT ALL PLANTS. ALL TREATMENT WAS IN OPERATION AT WS1. NAOH FEED WAS NOT IN OPERATION AT SF1 DUE TO ACCEPTABLE NATURAL PH. NO OTHER ISSUES NOTED. 6.DISTRIBUTION CONSISTS OF 201 SINGLE FAMILY RESIDENTIAL CONNECTIONS. NO ISSUES NOTED. 7.STORAGE CONSISTS OF TWO HYDROPNEUMATIC TANKS, 4,000 GALLONS AND 10,000 GALLONS. DOL INSPECTION WAS EXPIRED ON BOTH. NO OTHER ISSUES NOTED. 8.STAGE 2 DBP IS APPROVED. LC/TCR PLANS WERE NOT REVIEWED.",2015-08-19,2021-06-29
2626,2021Q2,NC0425010,85,2015-06-24,ST,SNSV,N,N,N,N,N,N,N,N,N,N,N,"DALTON GASKINS, JON HAMMOND, PAM FRENCH~ WATER SYSTEM PROVIDED PUBLIC WATER SUPPLY WITH A LIST OF BACKFLOW PREVENTION DEVICES OWNED BY THE SYSTEM. I ADVISED HIM THAT IT IS NECESSARY TO ALSO COMPOSE A LIST OF ALL BACKFLOW PREVENTION DEVICES IN THE SYSTEM AND THEIR TESTING SCHEDULE. -ALL COMPLIANCE PLANS AND OPERATION MANUALS IN ORDER; ADVISED DALTON THAT WHEN TAKING SAMPLES FROM DEDICATED SAMPLING STATIONS, NONE OF THESE SPECIMENS MAY BE INVALIDATED. -SEVERAL WELLS WERE LEAKING FROM, AND/OR MISSING SCREENS ON, THE AIR-RELIEF VALVE BLOW-OFF. THESE AIR-RELIEF VALVES WILL BE DISMANTLED AND CLEANED. -AMDRO BRAND FIRE ANT KILLER WAS BEING APPLIED ON ANT-HILLS ON THE WELL SITES- I ADVISED THE WATER SYSTEM TO STOP APPLYING THIS PESTICIDES ON THE WELL-SITES IMMEDIATELY. -THE WATER SYSTEM HAS ALMOST COMPLETELY LOST ACCESS TO THREE OF THE WELL SITES DUE TO CONSTRUCTION. COVE CITY WELLS #1 AND #2 DOWN BECAUSE GRUNDFOS PUMPS INCAPACITATED. -CCRS TO BE PUBLISHED/DISTRIBUTED JUNE 30,2015, CHLORINE CONVERSION TO COMMENCE JULY 17, 2015.",2015-08-19,2021-06-29
2751,2021Q2,NC0241139,29,2015-06-11,ST,SNSV,N,N,N,N,N,N,N,N,N,N,N,"M

In [72]:
len(site_visits_df.columns)

20

### Web Scraping Column description and data type

In [73]:
def get_html():
    URL = "https://echo.epa.gov/tools/data-downloads/sdwa-download-summary"
    r = requests.get(URL)  
    html_soup = BeautifulSoup(r.content, 'html.parser') 
    return html_soup
html_soup = get_html()

In [74]:
cols = []
for th in html_soup.find_all("div", {"class": 'field-item even'})[0].find_all('h3')[-2].find_next_sibling('table').find("thead").find_all("th"):
    cols.append(th.get_text().replace(u'\xa0', u''))
rows = []
for row in html_soup.find_all("div", {"class": 'field-item even'})[0].find_all('h3')[-2].find_next_sibling('table').find("tbody").find_all("tr"):
    r = []
    for td in row.find_all('td'):
        r.append(td.get_text().replace(u'\xa0', u''))
    rows.append(r)
column_datatype = pd.DataFrame(rows, columns=cols)
column_datatype = column_datatype[column_datatype.Element.isin(site_visits_df.columns)]
column_datatype

,Element,Data Type,Length
0,PWSID,Char,9
1,SUBMISSIONYEARQUARTER,Char,6
2,VISIT_ID,Char,
3,VISIT_DATE,Date,
4,AGENCY_TYPE_CODE,Char,4
5,VISIT_REASON_CODE,Char,4
6,MANAGEMENT_OPS_EVAL_CODE,Char,1
7,SOURCE_WATER_EVAL_CODE,Char,1
8,SECURITY_EVAL_CODE,Char,1
9,PUMPS_EVAL_CODE,Char,1


In [75]:
data_dictionary = {
    p.find_all('strong')[0].getText().replace(u'\xa0', u'').replace('VISIT_DATE (aka SITE_VISIT_DATE)', 'VISIT_DATE') : p.getText().replace(u'\xa0', u' ').replace('VISIT_DATE (aka SITE_VISIT_DATE)', 'VISIT_DATE') 
    for p in html_soup.find_all("div", {"class": 'field-item even'})[0].find_all('p') 
    if p.find_all('strong') and len(p.find_all('strong')) == 1 and p.find_all('strong')[0].getText().replace(u'\xa0', u'') in list(site_visits_df.columns) + ['VISIT_DATE (aka SITE_VISIT_DATE)']
}
pd.set_option('display.max_colwidth', None)
col_desc = pd.DataFrame([data_dictionary[col] for col in data_dictionary], columns=['raw_desc'])

col_desc[['COLUMN', 'DESCRIPTION']] = col_desc['raw_desc'].str.split('-', 1, expand=True)
col_desc = col_desc[['COLUMN', 'DESCRIPTION']]
col_desc

,COLUMN,DESCRIPTION
0,AGENCY_TYPE_CODE,The agency type that conducted the site visit.
1,COMPLIANCE_EVAL_CODE,Compliance evaluation results from site visit.
2,DATA_VERIFICATION_EVAL_CODE,Data verification evaluation results from site visit.
3,DISTRIBUTION_EVAL_CODE,Distribution system evaluation results from site visit.
4,FINANCIAL_EVAL_CODE,Financial evaluation results from site visit.
5,FINISHED_WATER_STOR_EVAL_CODE,Water storage evaluation from site visit.
6,FIRST_REPORTED_DATE,The first reported date for the milestone event. The date format is MM/DD/YYYY.
7,LAST_REPORTED_DATE,The last reported date for the milestone event. The date format is MM/DD/YYYY.
8,MANAGEMENT_OPS_EVAL_CODE,Management operations evaluation results from site visit.
9,OTHER_EVAL_CODE,Other facility equipment/management evaluation results from site visit.


### Generating create SQL statement to create 'site_visit' table to store site visit details

In [76]:
temp_dict = {}

def space(n):
    return ' ' * (max([len(col) for col in site_visits_df.columns]) + 5 - n)

for ind in column_datatype.index:
    col_name = column_datatype['Element'][ind]
    data_type = column_datatype['Data Type'][ind]
    length = column_datatype['Length'][ind]
    data_type = 'VARCHAR' if data_type == 'Char' else 'DATE' if data_type == 'Date' else 'INT'
    data_type = data_type + '('+ length +')' if length != '' and data_type == 'VARCHAR' else data_type + '(255)' if data_type == 'VARCHAR' else data_type
    temp_dict[col_name] = data_type
print('CREATE TABLE SITE_VISIT (')
for col in site_visits_df.columns:
    print(f'\t{col}{space(len(col))}{temp_dict[col]}', end='')
    if col != site_visits_df.columns[-1]:
        print(',')
    else:
        print()
print(') ENGINE = InnoDB;')

CREATE TABLE SITE_VISIT (
	SUBMISSIONYEARQUARTER             VARCHAR(6),
	PWSID                             VARCHAR(9),
	VISIT_ID                          VARCHAR(255),
	VISIT_DATE                        DATE,
	AGENCY_TYPE_CODE                  VARCHAR(4),
	VISIT_REASON_CODE                 VARCHAR(4),
	MANAGEMENT_OPS_EVAL_CODE          VARCHAR(1),
	SOURCE_WATER_EVAL_CODE            VARCHAR(1),
	SECURITY_EVAL_CODE                VARCHAR(1),
	PUMPS_EVAL_CODE                   VARCHAR(1),
	OTHER_EVAL_CODE                   VARCHAR(1),
	COMPLIANCE_EVAL_CODE              VARCHAR(1),
	DATA_VERIFICATION_EVAL_CODE       VARCHAR(1),
	TREATMENT_EVAL_CODE               VARCHAR(1),
	FINISHED_WATER_STOR_EVAL_CODE     VARCHAR(1),
	DISTRIBUTION_EVAL_CODE            VARCHAR(1),
	FINANCIAL_EVAL_CODE               VARCHAR(1),
	VISIT_COMMENTS                    VARCHAR(255),
	FIRST_REPORTED_DATE               DATE,
	LAST_REPORTED_DATE                DATE
) ENGINE = InnoDB;


In [84]:
columns = site_visits_df.columns.to_list()

date_columns = ['VISIT_DATE', 'FIRST_REPORTED_DATE', 'LAST_REPORTED_DATE']

print('''
LOAD DATA INFILE 'C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/SDWA_SITE_VISITS.csv'
INTO TABLE SITE_VISIT
FIELDS TERMINATED BY ','
ESCAPED BY ''
OPTIONALLY ENCLOSED BY '"'
LINES TERMINATED BY '\\n'
IGNORE 1 ROWS
''', end='')
print('(', end='')
for col in columns:
    print(f'@{col}', end='')
    if col != columns[-1]:
        print(',', end='')
print(')')
print('SET')
for col in columns:
    if col in date_columns:
        print(f'{col} = IF(@{col} = \'\', NULL, STR_TO_DATE(@{col}, \'%m/%d/%Y\')),')
    else:
        print(f'{col} = IF(@{col} = \'\', NULL, @{col}),')


LOAD DATA INFILE 'C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/SDWA_SITE_VISITS.csv'
INTO TABLE SITE_VISIT
FIELDS TERMINATED BY ','
ESCAPED BY ''
OPTIONALLY ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 ROWS
(@SUBMISSIONYEARQUARTER,@PWSID,@VISIT_ID,@VISIT_DATE,@AGENCY_TYPE_CODE,@VISIT_REASON_CODE,@MANAGEMENT_OPS_EVAL_CODE,@SOURCE_WATER_EVAL_CODE,@SECURITY_EVAL_CODE,@PUMPS_EVAL_CODE,@OTHER_EVAL_CODE,@COMPLIANCE_EVAL_CODE,@DATA_VERIFICATION_EVAL_CODE,@TREATMENT_EVAL_CODE,@FINISHED_WATER_STOR_EVAL_CODE,@DISTRIBUTION_EVAL_CODE,@FINANCIAL_EVAL_CODE,@VISIT_COMMENTS,@FIRST_REPORTED_DATE,@LAST_REPORTED_DATE)
SET
SUBMISSIONYEARQUARTER = IF(@SUBMISSIONYEARQUARTER = '', NULL, @SUBMISSIONYEARQUARTER),
PWSID = IF(@PWSID = '', NULL, @PWSID),
VISIT_ID = IF(@VISIT_ID = '', NULL, @VISIT_ID),
VISIT_DATE = IF(@VISIT_DATE = '', NULL, STR_TO_DATE(@VISIT_DATE, '%m/%d/%Y')),
AGENCY_TYPE_CODE = IF(@AGENCY_TYPE_CODE = '', NULL, @AGENCY_TYPE_CODE),
VISIT_REASON_CODE = IF(@VISIT_REASON_CODE = '', NULL, @VI